# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [29]:
using Manifolds
using Manopt
using LinearAlgebra
using Random
using Plots
using LaTeXStrings

In [30]:
include("../../../src/decompositions/signals/naive_low_rank_approximation.jl")
include("../../../src/decompositions/signals/curvature_corrected_low_rank_approximation.jl")
include("../../../src/decompositions/signals/exact_low_rank_approximation.jl")
include("../../../src/decompositions/signals/stochastic_curvature_corrected_low_rank_approximation.jl")

include("../../../src/functions/loss_functions/curvature_corrected_loss.jl")
include("../../../src/functions/loss_functions/exact_loss.jl")
include("../../../src/functions/jacobi_field/beta.jl")

β (generic function with 1 method)

In [31]:
black = RGBA{Float64}(colorant"#000000")
TolVibrantOrange = RGBA{Float64}(colorant"#EE7733")
TolVibrantBlue = RGBA{Float64}(colorant"#0077BB")
TolVibrantTeal = RGBA{Float64}(colorant"#009988")
TolVibrantMagenta = RGBA{Float64}(colorant"#EE3377")
TolVibrantCyan = RGBA{Float64}(colorant"#33BBEE");
render_size = 1

1

### Load data and construct manifold ###

In [32]:
# load data
M = Sphere(6)
d = manifold_dimension(M)
n = 100  # 100


100

In [33]:
θ = collect(range(0, 2 * π, n+1))[1:n]
predata = [[cos(θ[i]), sin(θ[i]), 0., 0., 0., 0., 0.] for i in 1:n]
σ = .05  # variance

Random.seed!(31)
data = [exp(M, predata[i], random_tangent(M, predata[i], Val(:Gaussian), σ)) for i in 1:n]; # ∈ S(6)^n

In [34]:
# Export slice image
# asymptote_export_S2_signals("results/artificial1D_orig.asy", points=[data],
# colors=Dict(:points => [TolVibrantBlue]),
# dot_size=3.5,
# camera_position=(1.0, 0.5, 0.5)); 

### Construct low rank approximation ###

In [35]:
q₁ = mean(M, data)
println(q₁)
log_q_data₁ = log.(Ref(M), Ref(q₁), data);  # ∈ T_q P(3)^n
# TODO compute theoretical lower bound
ONB₁ = get_basis.(Ref(M), Ref(q₁), DiagonalizingOrthonormalBasis.(log_q_data₁))
κ₁ = maximum([maximum(distance(M, q₁, data[i])^2 .* ONB₁[i].data.eigenvalues) for i in 1:n])
println(κ₁)


summed_pairwise_distances = [sum([distance.(Ref(M), Ref(data[i]), data) .^2]) for i in 1:n]
q₂ = data[argmin(summed_pairwise_distances)]
log_q_data₂ = log.(Ref(M), Ref(q₂), data);  # ∈ T_q P(3)^n
println(q₂)
ONB₂ = get_basis.(Ref(M), Ref(q₂), DiagonalizingOrthonormalBasis.(log_q_data₂))
κ₂ = maximum([maximum(distance(M, q₂, data[i])^2 .* ONB₂[i].data.eigenvalues) for i in 1:n])
println(κ₂)

[0.011362771023773281, -0.014665511087507247, 0.32104150872364784, -0.4013489917107375, -0.07156976421128587, -0.852341865783546, -0.06243604273002383]
2.870971878384671
[0.9944358300450549, -0.05035588364543773, 0.05181675794418579, 0.03470388913095954, 0.013728091849230824, 0.009286850030275141, -0.06631457223660438]
9.16012544722712


In [36]:
# stochastic_curvature_corrected_low_rank_approximation(M, q₂, data, 1; max_iter=10); 

In [37]:
exact_low_rank_approximation(M, q₁, data, 1; stepsize=1/20, max_iter=20); 

Initial  F(x): 0.40744920354 | 
# 1     change: 0.184199801 |  F(x): 0.40477816649 | 
# 2     change: 0.168096579 |  F(x): 0.40255374945 | 
# 3     change: 0.153393542 |  F(x): 0.40070144402 | 
# 4     change: 0.139970685 |  F(x): 0.39915911961 | 
# 5     change: 0.127717904 |  F(x): 0.39787498234 | 
# 6     change: 0.116534275 |  F(x): 0.39680586609 | 
# 7     change: 0.106327345 |  F(x): 0.39591580299 | 
# 8     change: 0.097012480 |  F(x): 0.39517482849 | 
# 9     change: 0.088512241 |  F(x): 0.39455798330 | 
# 10    change: 0.080755800 |  F(x): 0.39404448008 | 
# 11    change: 0.073678401 |  F(x): 0.39361700809 | 
# 12    change: 0.067220861 |  F(x): 0.39326115320 | 
# 13    change: 0.061329100 |  F(x): 0.39296491415 | 
# 14    change: 0.055953713 |  F(x): 0.39271829944 | 
# 15    change: 0.051049579 |  F(x): 0.39251299126 | 
# 16    change: 0.046575491 |  F(x): 0.39234206546 | 
# 17    change: 0.042493823 |  F(x): 0.39219975836 | 
# 18    change: 0.038770227 |  F(x): 0.39208127237

In [38]:
exact_low_rank_approximation(M, q₂, data, 1; stepsize=1/100, max_iter=20); 

Initial  F(x): 0.04339286801 | 
# 1     change: 0.025271948 |  F(x): 0.04319770468 | 
# 2     change: 0.024770628 |  F(x): 0.04301021233 | 
# 3     change: 0.024279290 |  F(x): 0.04283008908 | 
# 4     change: 0.023797735 |  F(x): 0.04265704494 | 
# 5     change: 0.023325770 |  F(x): 0.04249080134 | 
# 6     change: 0.022863203 |  F(x): 0.04233109065 | 
# 7     change: 0.022409850 |  F(x): 0.04217765578 | 
# 8     change: 0.021965525 |  F(x): 0.04203024978 | 
# 9     change: 0.021530051 |  F(x): 0.04188863539 | 
# 10    change: 0.021103253 |  F(x): 0.04175258471 | 
# 11    change: 0.020684956 |  F(x): 0.04162187879 | 
# 12    change: 0.020274994 |  F(x): 0.04149630732 | 
# 13    change: 0.019873201 |  F(x): 0.04137566824 | 
# 14    change: 0.019479414 |  F(x): 0.04125976747 | 
# 15    change: 0.019093475 |  F(x): 0.04114841856 | 
# 16    change: 0.018715229 |  F(x): 0.04104144238 | 
# 17    change: 0.018344521 |  F(x): 0.04093866689 | 
# 18    change: 0.017981204 |  F(x): 0.04083992677

In [39]:
max_iter = 50

nR_q₁ = []
nU₁ = []
ccR_q₁ = []
ccU₁ = []
eR_q₁ = []
eU₁ = []

nR_q₂ = []
nU₂ = []
ccR_q₂ = []
ccU₂ = []
eR_q₂ = []
eU₂ = []

for i in 1:d
    println("#$(i) | computing naive low-rank approximation")
    nRr_q₁, nUr₁ = naive_low_rank_approximation(M, q₁, data, i)
    push!(nR_q₁, nRr_q₁)
    push!(nU₁, nUr₁)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₁, ccUr₁ = curvature_corrected_low_rank_approximation(M, q₁, data, i); 
    push!(ccR_q₁, ccRr_q₁)
    push!(ccU₁, ccUr₁)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q₁, eUr₁ = exact_low_rank_approximation(M, q₁, data, i; stepsize=1/50, max_iter=max_iter); 
    push!(eR_q₁, eRr_q₁)
    push!(eU₁, eUr₁)

    println("#$(i) | computing naive low-rank approximation")
    nRr_q₂, nUr₂ = naive_low_rank_approximation(M, q₂, data, i)
    push!(nR_q₂, nRr_q₂)
    push!(nU₂, nUr₂)
    println("#$(i) | computing curvature corrected low-rank approximation")
    ccRr_q₂, ccUr₂ = curvature_corrected_low_rank_approximation(M, q₂, data, i); 
    push!(ccR_q₂, ccRr_q₂)
    push!(ccU₂, ccUr₂)
    println("#$(i) | computing exact low-rank approximation")
    eRr_q₂, eUr₂ = exact_low_rank_approximation(M, q₂, data, i; stepsize=1/100, max_iter=max_iter); 
    push!(eR_q₂, eRr_q₂)
    push!(eU₂, eUr₂)
end

#1 | computing naive low-rank approximation
#1 | computing curvature corrected low-rank approximation
#1 | computing exact low-rank approximation
Initial  F(x): 0.40744920354 | 
# 1     change: 0.073679921 |  F(x): 0.40635149010 | 
# 2     change: 0.071103867 |  F(x): 0.40532919104 | 
# 3     change: 0.068617327 |  F(x): 0.40437713995 | 
# 4     change: 0.066217243 |  F(x): 0.40349052243 | 
# 5     change: 0.063900656 |  F(x): 0.40266485239 | 
# 6     change: 0.061664705 |  F(x): 0.40189594981 | 
# 7     change: 0.059506623 |  F(x): 0.40117992010 | 
# 8     change: 0.057423733 |  F(x): 0.40051313473 | 
# 9     change: 0.055413449 |  F(x): 0.39989221320 | 
# 10    change: 0.053473269 |  F(x): 0.39931400622 | 
# 11    change: 0.051600775 |  F(x): 0.39877558001 | 
# 12    change: 0.049793631 |  F(x): 0.39827420163 | 
# 13    change: 0.048049578 |  F(x): 0.39780732529 | 
# 14    change: 0.046366434 |  F(x): 0.39737257968 | 
# 15    change: 0.044742091 |  F(x): 0.39696775599 | 
# 16    chan

In [40]:
# basepoint q₁
ref_distance₁ = sum(distance.(Ref(M), Ref(q₁), data).^2)

naive_tangent_distances_r₁ = zeros(d)
predicted_naive_distances_r₁ = zeros(d)
true_naive_distances_r₁ = zeros(d)

curvature_corrected_tangent_distances_r₁ = zeros(d)
predicted_curvature_corrected_distances_r₁ = zeros(d)
true_curvature_corrected_distances_r₁ = zeros(d)

exact_tangent_distances_r₁ = zeros(d)
exact_distances_r₁ = zeros(d)

# basepoint q₂
ref_distance₂ = sum(distance.(Ref(M), Ref(q₂), data).^2)

naive_tangent_distances_r₂ = zeros(d)
predicted_naive_distances_r₂ = zeros(d)
true_naive_distances_r₂ = zeros(d)

curvature_corrected_tangent_distances_r₂ = zeros(d)
predicted_curvature_corrected_distances_r₂ = zeros(d)
true_curvature_corrected_distances_r₂ = zeros(d)

exact_tangent_distances_r₂ = zeros(d)
exact_distances_r₂ = zeros(d)

for rank in 1:d
    ## basepoint q₁
    naive_log_q_data_r₁ = [sum([nR_q₁[rank][i] * nU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₁ = [sum([ccR_q₁[rank][i] * ccU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₁ = [sum([eR_q₁[rank][i] * eU₁[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # expoentiate back
    naive_data_r₁ = exp.(Ref(M), Ref(q₁), naive_log_q_data_r₁)
    curvature_corrected_data_r₁ = exp.(Ref(M), Ref(q₁), curvature_corrected_log_q_data_r₁)
    exact_data_r₁ = exp.(Ref(M), Ref(q₁), exact_log_q_data_r₁)


    # compute relative tangent space error
    naive_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - naive_log_q_data_r₁).^2) / ref_distance₁
    curvature_corrected_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - curvature_corrected_log_q_data_r₁).^2) / ref_distance₁
    exact_tangent_distances_r₁[rank] = sum(norm.(Ref(M), Ref(q₁),  log_q_data₁ - exact_log_q_data_r₁).^2) / ref_distance₁


    # compute relative manifold error
    predicted_naive_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, naive_log_q_data_r₁)
    true_naive_distances_r₁[rank] = exact_loss(M, q₁, data, naive_log_q_data_r₁)
    predicted_curvature_corrected_distances_r₁[rank] = curvature_corrected_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    true_curvature_corrected_distances_r₁[rank] = exact_loss(M, q₁, data, curvature_corrected_log_q_data_r₁)
    exact_distances_r₁[rank] = exact_loss(M, q₁, data, exact_log_q_data_r₁)

    ## basepoint q₂
    naive_log_q_data_r₂ = [sum([nR_q₂[rank][i] * nU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    curvature_corrected_log_q_data_r₂ = [sum([ccR_q₂[rank][i] * ccU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    exact_log_q_data_r₂ = [sum([eR_q₂[rank][i] * eU₂[rank][k,i] for i in 1:rank]) for k in 1:n]
    
    # exponentiate back
    naive_data_r₂ = exp.(Ref(M), Ref(q₂), naive_log_q_data_r₂)
    curvature_corrected_data_r₂ = exp.(Ref(M), Ref(q₂), curvature_corrected_log_q_data_r₂)
    exact_data_r₂ = exp.(Ref(M), Ref(q₂), exact_log_q_data_r₂)


    # compute relative tangent space error
    naive_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - naive_log_q_data_r₂).^2) / ref_distance₂
    curvature_corrected_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - curvature_corrected_log_q_data_r₂).^2) / ref_distance₂
    exact_tangent_distances_r₂[rank] = sum(norm.(Ref(M), Ref(q₂),  log_q_data₂ - exact_log_q_data_r₂).^2) / ref_distance₂


    # compute relative manifold error
    predicted_naive_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, naive_log_q_data_r₂)
    true_naive_distances_r₂[rank] = exact_loss(M, q₂, data, naive_log_q_data_r₂)
    predicted_curvature_corrected_distances_r₂[rank] = curvature_corrected_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    true_curvature_corrected_distances_r₂[rank] = exact_loss(M, q₂, data, curvature_corrected_log_q_data_r₂)
    exact_distances_r₂[rank] = exact_loss(M, q₂, data, exact_log_q_data_r₂)
end

In [41]:
# We want plots with (1) the lower bound error, (2) the actually uncorrected manifold error and (3) the corrected manifold error
# basepoint q₁
plot(1:d, [β(κ₁)^2 .* naive_tangent_distances_r₁, true_naive_distances_r₁, true_curvature_corrected_distances_r₁, exact_distances_r₁], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_1.svg")
plot(1:d, [β(κ₁)^2 .* naive_tangent_distances_r₁ .+ 1e-16, true_naive_distances_r₁ .+ 1e-16, true_curvature_corrected_distances_r₁ .+ 1e-16, exact_distances_r₁ .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_1.svg")
# basepoint q₂
plot(1:d, [β(κ₂)^2 .* naive_tangent_distances_r₂, true_naive_distances_r₂, true_curvature_corrected_distances_r₂, exact_distances_r₂], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(0,1), xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$"))
savefig("results/artificial1D_errors_by_rank_2.svg")
plot(1:d, [β(κ₂)^2 .* naive_tangent_distances_r₂ .+ 1e-16, true_naive_distances_r₂ .+ 1e-16, true_curvature_corrected_distances_r₂ .+ 1e-16, exact_distances_r₂ .+ 1e-16], label = ["theoretical lower bound" "naive" "curvature corrected" "exact"], ylims=(1e-16,1), xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\varepsilon_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logerrors_by_rank_2.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_logerrors_by_rank_2.svg"

In [42]:
# It would be nice to also have a plot that tells us something about the error in predicting the manifold loss (using CCL) and the actual loss 
# (1) for the naive approach (2) for the curvature corrected approach
# basepoint q₁
plot(1:d, [(predicted_naive_distances_r₁ .+ 1e-16) ./ (true_naive_distances_r₁ .+ 1e-16), (predicted_curvature_corrected_distances_r₁ .+ 1e-16) ./ (true_curvature_corrected_distances_r₁ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank_1.svg")
plot(1:d, [(predicted_naive_distances_r₁ .+ 1e-16) ./ (true_naive_distances_r₁ .+ 1e-16), (predicted_curvature_corrected_distances_r₁ .+ 1e-16) ./ (true_curvature_corrected_distances_r₁ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank_1.svg")
# basepoint q₂
plot(1:d, [(predicted_naive_distances_r₂ .+ 1e-16) ./ (true_naive_distances_r₂ .+ 1e-16), (predicted_curvature_corrected_distances_r₂ .+ 1e-16) ./ (true_curvature_corrected_distances_r₂ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d),xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$"))
savefig("results/artificial1D_discrepancy_by_rank_2.svg")
plot(1:d, [(predicted_naive_distances_r₂ .+ 1e-16) ./ (true_naive_distances_r₂ .+ 1e-16), (predicted_curvature_corrected_distances_r₂ .+ 1e-16) ./ (true_curvature_corrected_distances_r₂ .+ 1e-16)], label = ["discrepancy in initialisation" "discrepancy in solutions"], xlims=(1,d), xaxis=("approximation rank"), yaxis=(L"$\delta_{rel}$", :log), legend=:bottomleft)
savefig("results/artificial1D_logdiscrepancy_by_rank_2.svg")

"/Users/wdiepeveen/Documents/PhD/Projects/8 - Manifold-valued tensor decomposition/src/manifold-valued-tensors/experiments/1D/S2/results/artificial1D_logdiscrepancy_by_rank_2.svg"